# Dzień 2 - Moduł 6: NLP w praktyce biznesowej

## Cele modułu:
- Tworzenie chatbotów i conversational AI
- Automatyzacja przetwarzania dokumentów
- Personalizacja komunikacji z klientami
- Ekstrakcja informacji z dokumentów biznesowych
- Implementacja rozwiązań produkcyjnych

In [ ]:
# Import bibliotek
import os
import json
import re
import pandas as pd
import numpy as np
from transformers import pipeline
from datetime import datetime
import matplotlib.pyplot as plt

print("✅ Biblioteki załadowane!")

## 6.1 Tworzenie chatbotów i conversational AI

### Rodzaje chatbotów:

#### 1. Rule-based Chatbots
- Działają według predefiniowanych reguł
- Szybkie, przewidywalne
- Ograniczona elastyczność
- Dobre dla FAQ i prostych scenariuszy

#### 2. Retrieval-based Chatbots
- Wybierają odpowiedź z bazy gotowych odpowiedzi
- Średnia elastyczność
- Wymagają bazy wiedzy

#### 3. Generative Chatbots
- Generują odpowiedzi od zera
- Maksymalna elastyczność
- Wymagają dużych modeli (GPT, LLaMA)
- Mogą generować błędne informacje

### Komponenty chatbota:
1. **Intent Classification** - rozpoznanie intencji użytkownika
2. **Entity Extraction** - wydobycie kluczowych informacji
3. **Dialogue Management** - zarządzanie rozmową
4. **Response Generation** - tworzenie odpowiedzi
5. **Context Tracking** - śledzenie kontekstu rozmowy

In [ ]:
# Przykład 1: Prosty Rule-based Chatbot
class SimpleRuleBot:
    def __init__(self):
        self.rules = {
            r'.*\b(cześć|hej|witaj)\b.*': [
                "Cześć! Jak mogę Ci pomóc?",
                "Witaj! W czym mogę pomóc?"
            ],
            r'.*\b(godziny|otwarcie|otwarte)\b.*': [
                "Jesteśmy otwarci od poniedziałku do piątku, 9:00-17:00."
            ],
            r'.*\b(cena|koszt|ile kosztuje)\b.*': [
                "Nasze ceny zaczynają się od 99 zł. Mogę podać szczegóły dla konkretnego produktu."
            ],
            r'.*\b(kontakt|telefon|email)\b.*': [
                "Możesz się z nami skontaktować: telefon: 123-456-789, email: kontakt@firma.pl"
            ],
            r'.*\b(dziękuję|dzięki|thx)\b.*': [
                "Nie ma za co! Czy mogę pomóc w czymś jeszcze?",
                "Cieszę się, że mogłem pomóc!"
            ],
            r'.*\b(pa|żegnaj|do widzenia)\b.*': [
                "Do widzenia! Miłego dnia!",
                "Pa! Zapraszam ponownie!"
            ],
        }
        self.default_response = "Przepraszam, nie rozumiem. Czy możesz przeformułować pytanie?"
    
    def get_response(self, message):
        message = message.lower()
        for pattern, responses in self.rules.items():
            if re.match(pattern, message):
                return np.random.choice(responses)
        return self.default_response

# Test
print("=== RULE-BASED CHATBOT ===")
bot = SimpleRuleBot()

test_messages = [
    "Cześć!",
    "Jakie są godziny otwarcia?",
    "Ile to kosztuje?",
    "Jak mogę się skontaktować?",
    "Dziękuję!",
    "Co z pogodą?",  # nieznane pytanie
]

for msg in test_messages:
    response = bot.get_response(msg)
    print(f"\n👤 User: {msg}")
    print(f"🤖 Bot: {response}")

In [ ]:
# Przykład 2: Intent Classification Bot (ML-based)
print("\n=== INTENT CLASSIFICATION CHATBOT ===")

# Użyjemy zero-shot classification do rozpoznawania intencji
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Definiujemy możliwe intencje
intents = [
    "greeting",
    "ask_about_price",
    "ask_about_hours",
    "ask_about_product",
    "complaint",
    "farewell"
]

# Baza odpowiedzi dla intencji
intent_responses = {
    "greeting": "Witaj! Jak mogę Ci pomóc dzisiaj?",
    "ask_about_price": "Nasze ceny są konkurencyjne. Która kategoria produktów Cię interesuje?",
    "ask_about_hours": "Obsługujemy klientów od 9:00 do 17:00, poniedziałek-piątek.",
    "ask_about_product": "Chętnie opowiem o naszych produktach. Co Cię szczególnie interesuje?",
    "complaint": "Przykro mi słyszeć o problemie. Czy możesz podać więcej szczegółów?",
    "farewell": "Do widzenia! Dziękuję za rozmowę!"
}

def intent_bot(message):
    # Klasyfikacja intencji
    result = classifier(message, intents)
    detected_intent = result['labels'][0]
    confidence = result['scores'][0]
    
    print(f"\n👤 User: {message}")
    print(f"🧠 Detected Intent: {detected_intent} (confidence: {confidence:.2%})")
    print(f"🤖 Bot: {intent_responses.get(detected_intent, 'Nie rozumiem...')}")
    return detected_intent, confidence

# Test
test_messages = [
    "Hello there!",
    "How much does it cost?",
    "What are your business hours?",
    "I'm not satisfied with the service",
    "Goodbye!"
]

for msg in test_messages:
    intent_bot(msg)

In [ ]:
# Przykład 3: Conversational Bot z kontekstem
class ConversationalBot:
    def __init__(self):
        self.context = {}
        self.conversation_history = []
        print("🤖 Bot zainicjalizowany z pamięcią kontekstu")
    
    def add_message(self, role, message):
        self.conversation_history.append({
            'role': role,
            'message': message,
            'timestamp': datetime.now().isoformat()
        })
    
    def extract_info(self, message):
        """Ekstrakcja informacji z wiadomości"""
        # Prosta ekstrakcja nazwy
        name_match = re.search(r'(?:nazywam się|jestem|to)\s+([A-Z][a-z]+)', message)
        if name_match:
            self.context['name'] = name_match.group(1)
        
        # Ekstrakcja emaila
        email_match = re.search(r'([a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,})', message)
        if email_match:
            self.context['email'] = email_match.group(1)
    
    def get_response(self, user_message):
        self.add_message('user', user_message)
        self.extract_info(user_message)
        
        # Logika odpowiedzi z kontekstem
        if 'name' in self.context and len(self.conversation_history) == 1:
            response = f"Miło mi Cię poznać, {self.context['name']}! W czym mogę pomóc?"
        elif 'email' in self.context:
            response = f"Dziękuję! Zapisałem Twój email: {self.context['email']}"
        elif any(word in user_message.lower() for word in ['pomoc', 'problem', 'pytanie']):
            response = "Oczywiście! Opisz proszę swój problem, a postaram się pomóc."
        else:
            response = "Rozumiem. Czy mogę w czymś jeszcze pomóc?"
        
        self.add_message('bot', response)
        return response
    
    def get_conversation_summary(self):
        return {
            'total_messages': len(self.conversation_history),
            'context': self.context,
            'history': self.conversation_history
        }

# Test
print("\n=== CONVERSATIONAL BOT Z KONTEKSTEM ===")
conv_bot = ConversationalBot()

conversation = [
    "Cześć, nazywam się Anna",
    "Mam pytanie dotyczące produktu",
    "Mój email to anna@example.com",
    "Dziękuję za pomoc!"
]

for msg in conversation:
    print(f"\n👤 User: {msg}")
    response = conv_bot.get_response(msg)
    print(f"🤖 Bot: {response}")

# Podsumowanie
print("\n" + "="*70)
print("PODSUMOWANIE ROZMOWY:")
summary = conv_bot.get_conversation_summary()
print(f"Liczba wiadomości: {summary['total_messages']}")
print(f"Zebrane informacje: {summary['context']}")

## 6.2 Automatyzacja przetwarzania dokumentów

### Przypadki użycia:
- **Przetwarzanie faktur** - ekstrakcja kwot, dat, kontrahentów
- **Analiza CV** - wydobycie umiejętności, doświadczenia
- **Klasyfikacja dokumentów** - sortowanie według typu
- **Ekstrakcja klauzul** - dokumenty prawne
- **Podsumowania raportów** - automatyczne streszczenia

### Pipeline przetwarzania dokumentów:
1. **OCR** (jeśli dokument nie jest tekstowy)
2. **Preprocessing** - czyszczenie tekstu
3. **NER** - rozpoznawanie kluczowych encji
4. **Classification** - kategoryzacja
5. **Information Extraction** - ekstrakcja danych strukturalnych
6. **Validation** - weryfikacja poprawności
7. **Storage** - zapis do bazy danych

In [ ]:
# Przykład 1: Ekstrakcja informacji z faktur
print("=== AUTOMATYZACJA PRZETWARZANIA FAKTUR ===")

class InvoiceProcessor:
    def __init__(self):
        self.ner_pipeline = pipeline("ner", model="dslim/bert-base-NER", grouped_entities=True)
    
    def extract_invoice_info(self, text):
        """Ekstrakcja kluczowych informacji z faktury"""
        info = {
            'invoice_number': None,
            'date': None,
            'amount': None,
            'entities': []
        }
        
        # Numer faktury
        invoice_match = re.search(r'Invoice\s+(?:No\.?|Number)?\s*[:#]?\s*(\w+[-/]?\w+)', text, re.IGNORECASE)
        if invoice_match:
            info['invoice_number'] = invoice_match.group(1)
        
        # Data
        date_match = re.search(r'Date:\s*(\d{1,2}[-/]\d{1,2}[-/]\d{2,4})', text, re.IGNORECASE)
        if date_match:
            info['date'] = date_match.group(1)
        
        # Kwota
        amount_match = re.search(r'Total[:\s]+\$?([\d,]+\.\d{2})', text, re.IGNORECASE)
        if amount_match:
            info['amount'] = amount_match.group(1)
        
        # Nazwane encje (firmy, osoby)
        entities = self.ner_pipeline(text)
        info['entities'] = [(e['word'], e['entity_group']) for e in entities]
        
        return info

# Test
sample_invoice = """
INVOICE

Invoice Number: INV-2024-001
Date: 24/10/2024

From: TechCorp Solutions
123 Business Street
New York, NY 10001

To: John Smith
ABC Company
456 Client Avenue
Boston, MA 02101

Description: Professional Services
Amount: $2,500.00
Tax: $250.00
Total: $2,750.00
"""

processor = InvoiceProcessor()
invoice_data = processor.extract_invoice_info(sample_invoice)

print("Wyekstraktowane dane:")
print(json.dumps(invoice_data, indent=2, ensure_ascii=False))

In [ ]:
# Przykład 2: Klasyfikacja i analiza CV
print("\n=== ANALIZA CV ===")

class CVAnalyzer:
    def __init__(self):
        self.classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
    
    def analyze_cv(self, cv_text):
        result = {
            'skills': self.extract_skills(cv_text),
            'experience_years': self.extract_experience(cv_text),
            'education': self.extract_education(cv_text),
            'category': self.classify_profession(cv_text)
        }
        return result
    
    def extract_skills(self, text):
        """Ekstrakcja umiejętności technicznych"""
        common_skills = [
            'Python', 'Java', 'JavaScript', 'SQL', 'Machine Learning',
            'Deep Learning', 'NLP', 'Data Science', 'AWS', 'Docker',
            'Git', 'React', 'Node.js', 'TensorFlow', 'PyTorch'
        ]
        found_skills = [skill for skill in common_skills if skill.lower() in text.lower()]
        return found_skills
    
    def extract_experience(self, text):
        """Oszacowanie lat doświadczenia"""
        years_match = re.search(r'(\d+)\+?\s*years?\s+(?:of\s+)?experience', text, re.IGNORECASE)
        if years_match:
            return int(years_match.group(1))
        return None
    
    def extract_education(self, text):
        """Ekstrakcja wykształcenia"""
        degrees = ['PhD', 'Master', 'Bachelor', 'MSc', 'BSc', 'MBA']
        found_degrees = [deg for deg in degrees if deg.lower() in text.lower()]
        return found_degrees
    
    def classify_profession(self, text):
        """Klasyfikacja zawodu"""
        professions = [
            'Data Scientist',
            'Software Engineer',
            'Machine Learning Engineer',
            'Product Manager',
            'Business Analyst'
        ]
        result = self.classifier(text[:500], professions)  # Pierwszy fragment CV
        return result['labels'][0]

# Test
sample_cv = """
John Doe
Data Scientist

SUMMARY
Experienced data scientist with 5 years of experience in machine learning and NLP.
Passionate about solving complex problems using AI and deep learning.

SKILLS
- Python, SQL, TensorFlow, PyTorch
- Machine Learning, Deep Learning, NLP
- AWS, Docker, Git

EDUCATION
Master of Science in Computer Science
University of Technology, 2018

EXPERIENCE
Senior Data Scientist, TechCorp (2021-Present)
Data Scientist, StartupXYZ (2019-2021)
"""

cv_analyzer = CVAnalyzer()
cv_analysis = cv_analyzer.analyze_cv(sample_cv)

print("Analiza CV:")
print(json.dumps(cv_analysis, indent=2))

## 6.3 Personalizacja komunikacji z klientami

### Zastosowania:
- **Email marketing** - personalizowane wiadomości
- **Rekomendacje produktów** - dopasowane opisy
- **Dynamiczne treści** - dostosowane do użytkownika
- **A/B testing** - różne wersje komunikatów

### Elementy personalizacji:
- Imię i dane klienta
- Historia zakupów
- Preferencje i zainteresowania
- Segment klienta
- Kontekst (pora dnia, lokalizacja)
- Poprzednie interakcje

In [ ]:
# Przykład: System personalizacji komunikacji
print("=== PERSONALIZACJA KOMUNIKACJI ===")

class PersonalizationEngine:
    def __init__(self):
        self.templates = {
            'welcome': {
                'new_customer': "Witaj {name}! Cieszymy się, że do nas dołączyłeś/aś. Jako nowy klient otrzymujesz 20% zniżki!",
                'returning': "Witaj ponownie, {name}! Miło Cię widzieć. Mamy dla Ciebie specjalną ofertę!",
                'vip': "Witamy z powrotem, {name}! Jako VIP klient, masz dostęp do ekskluzywnych produktów."
            },
            'product_recommendation': {
                'based_on_history': "Cześć {name}! Skoro podobały Ci się {previous_products}, pomyślałem/am, że może zainteresuje Cię {recommended_product}.",
                'trending': "Hej {name}! Zobacz co jest teraz popularne: {trending_products}. Myślę, że to jest w Twoim stylu!",
                'personalized': "{name}, mamy coś specjalnego dla Ciebie: {product}. Pasuje do Twoich preferencji!"
            },
            'abandoned_cart': {
                'gentle': "{name}, zauważyliśmy że pozostawiłeś/aś produkty w koszyku. Czy mogę w czymś pomóc?",
                'urgent': "{name}! Produkty w Twoim koszyku są nadal dostępne, ale zapasy się kończą!",
                'discount': "{name}, mamy dla Ciebie specjalną zniżkę 15% na produkty w koszyku. Kod: SAVE15"
            }
        }
    
    def generate_message(self, customer_data, message_type, sub_type):
        """Generuje spersonalizowaną wiadomość"""
        template = self.templates.get(message_type, {}).get(sub_type, "Cześć {name}!")
        return template.format(**customer_data)
    
    def analyze_customer_segment(self, customer):
        """Określa segment klienta"""
        if customer['orders_count'] == 0:
            return 'new_customer'
        elif customer['total_spent'] > 1000:
            return 'vip'
        else:
            return 'returning'

# Przykładowi klienci
customers = [
    {
        'name': 'Anna',
        'orders_count': 0,
        'total_spent': 0,
        'previous_products': [],
    },
    {
        'name': 'Marek',
        'orders_count': 5,
        'total_spent': 500,
        'previous_products': ['Laptop', 'Mysz'],
        'recommended_product': 'Klawiatura mechaniczna',
    },
    {
        'name': 'Ewa',
        'orders_count': 15,
        'total_spent': 2500,
        'previous_products': ['iPhone', 'MacBook'],
    }
]

engine = PersonalizationEngine()

print("\nSPERSONALIZOWANE WIADOMOŚCI POWITALNE:\n")
for customer in customers:
    segment = engine.analyze_customer_segment(customer)
    message = engine.generate_message(customer, 'welcome', segment)
    print(f"👤 {customer['name']} ({segment})")
    print(f"✉️  {message}")
    print()

In [ ]:
# Dynamiczne generowanie treści
print("\n=== DYNAMICZNE GENEROWANIE OPISÓW PRODUKTÓW ===")

class ProductDescriptionGenerator:
    def __init__(self):
        self.generator = pipeline("text-generation", model="gpt2")
    
    def generate_description(self, product_name, target_audience, style='professional'):
        """
        Generuje opis produktu dostosowany do grupy docelowej.
        """
        prompts = {
            'professional': f"Write a professional product description for {product_name} targeting {target_audience}:",
            'casual': f"Write a friendly product description for {product_name} that {target_audience} would love:",
            'luxury': f"Write an elegant, luxury product description for {product_name} for discerning {target_audience}:"
        }
        
        prompt = prompts.get(style, prompts['professional'])
        result = self.generator(prompt, max_length=100, num_return_sequences=1, temperature=0.7)
        return result[0]['generated_text']

# Test
desc_gen = ProductDescriptionGenerator()

products = [
    ('Smartwatch', 'tech enthusiasts', 'professional'),
    ('Running Shoes', 'athletes', 'casual'),
]

for product, audience, style in products:
    description = desc_gen.generate_description(product, audience, style)
    print(f"\n📦 Produkt: {product}")
    print(f"🎯 Grupa: {audience}")
    print(f"🎨 Styl: {style}")
    print(f"📝 Opis:\n{description[:200]}...")

## 6.4 Kompleksowy case study: System obsługi klienta

### Wymagania:
1. Automatyczna klasyfikacja zgłoszeń
2. Ekstrakcja kluczowych informacji
3. Routing do odpowiedniego działu
4. Automatyczne odpowiedzi dla prostych przypadków
5. Analiza sentymentu i priorytetyzacja
6. Generowanie raportów

In [ ]:
# Kompleksowy system obsługi klienta
class CustomerSupportSystem:
    def __init__(self):
        print("Inicjalizacja systemu obsługi klienta...")
        self.classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
        self.sentiment = pipeline("sentiment-analysis")
        self.qa = pipeline("question-answering")
        
        # Baza wiedzy
        self.knowledge_base = {
            'shipping': "Wysyłka trwa 3-5 dni roboczych. Darmowa dostawa przy zamówieniach powyżej 100 zł.",
            'returns': "Możesz zwrócić produkt w ciągu 30 dni od zakupu. Zwrot kosztów wysyłki po naszej stronie.",
            'payment': "Akceptujemy płatności kartą, przelewem i BLIK. Płatność jest w 100% bezpieczna.",
        }
        
        self.departments = {
            'technical_issue': 'IT Support',
            'billing_question': 'Finance',
            'product_inquiry': 'Sales',
            'complaint': 'Customer Service Manager',
            'general_question': 'Customer Service'
        }
        
        print("✅ System gotowy!\n")
    
    def process_ticket(self, ticket_text, customer_email):
        """
        Przetwarza zgłoszenie klienta.
        """
        print("="*70)
        print("PRZETWARZANIE ZGŁOSZENIA")
        print("="*70)
        
        ticket = {
            'customer_email': customer_email,
            'text': ticket_text,
            'timestamp': datetime.now().isoformat()
        }
        
        # 1. Klasyfikacja typu zgłoszenia
        print("\n1️⃣ KLASYFIKACJA:")
        categories = list(self.departments.keys())
        classification = self.classifier(ticket_text, categories)
        ticket['category'] = classification['labels'][0]
        ticket['confidence'] = classification['scores'][0]
        print(f"   Kategoria: {ticket['category']}")
        print(f"   Pewność: {ticket['confidence']:.2%}")
        
        # 2. Analiza sentymentu
        print("\n2️⃣ ANALIZA SENTYMENTU:")
        sentiment = self.sentiment(ticket_text)[0]
        ticket['sentiment'] = sentiment['label']
        ticket['sentiment_score'] = sentiment['score']
        emoji = "😊" if sentiment['label'] == 'POSITIVE' else "😞"
        print(f"   {emoji} {sentiment['label']} ({sentiment['score']:.2%})")
        
        # 3. Priorytetyzacja
        print("\n3️⃣ PRIORYTET:")
        if ticket['category'] == 'complaint' and ticket['sentiment'] == 'NEGATIVE':
            priority = 'HIGH'
        elif ticket['category'] == 'technical_issue':
            priority = 'MEDIUM'
        else:
            priority = 'NORMAL'
        ticket['priority'] = priority
        print(f"   Priority: {priority}")
        
        # 4. Routing
        print("\n4️⃣ ROUTING:")
        ticket['assigned_to'] = self.departments[ticket['category']]
        print(f"   Przypisano do: {ticket['assigned_to']}")
        
        # 5. Automatyczna odpowiedź (jeśli możliwe)
        print("\n5️⃣ AUTOMATYCZNA ODPOWIEDŹ:")
        auto_response = self.generate_auto_response(ticket)
        if auto_response:
            print(f"   ✅ {auto_response}")
            ticket['auto_response'] = auto_response
        else:
            print("   ⏸️  Wymaga ręcznej obsługi")
        
        return ticket
    
    def generate_auto_response(self, ticket):
        """Generuje automatyczną odpowiedź dla prostych przypadków."""
        if ticket['category'] == 'general_question':
            # Sprawdź bazę wiedzy
            text_lower = ticket['text'].lower()
            if 'wysyłka' in text_lower or 'dostawa' in text_lower:
                return self.knowledge_base['shipping']
            elif 'zwrot' in text_lower:
                return self.knowledge_base['returns']
            elif 'płatność' in text_lower:
                return self.knowledge_base['payment']
        return None

# Test systemu
print("\n=== TEST SYSTEMU OBSŁUGI KLIENTA ===")
support_system = CustomerSupportSystem()

# Przykładowe zgłoszenia
tickets = [
    {
        'text': "I'm having trouble logging into my account. The password reset doesn't work.",
        'email': 'user1@example.com'
    },
    {
        'text': "This product is terrible! I want a full refund immediately. Very disappointed!",
        'email': 'angry@example.com'
    },
    {
        'text': "How long does shipping usually take?",
        'email': 'curious@example.com'
    },
]

processed_tickets = []
for ticket_data in tickets:
    processed = support_system.process_ticket(ticket_data['text'], ticket_data['email'])
    processed_tickets.append(processed)
    print("\n")

In [ ]:
# Generowanie raportu
print("\n=== RAPORT Z PRZETWARZANIA ZGŁOSZEŃ ===")

df_tickets = pd.DataFrame(processed_tickets)

print(f"\nLiczba zgłoszeń: {len(df_tickets)}")
print(f"\nRozkład kategorii:")
print(df_tickets['category'].value_counts())

print(f"\nRozkład priorytetów:")
print(df_tickets['priority'].value_counts())

print(f"\nRozkład sentymentów:")
print(df_tickets['sentiment'].value_counts())

# Wizualizacja
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

df_tickets['category'].value_counts().plot(kind='bar', ax=axes[0], title='Kategorie')
df_tickets['priority'].value_counts().plot(kind='bar', ax=axes[1], title='Priorytety', color='orange')
df_tickets['sentiment'].value_counts().plot(kind='bar', ax=axes[2], title='Sentyment', color='green')

plt.tight_layout()
plt.show()

## Podsumowanie całego szkolenia

### Dzień 1 - Podstawy:
✅ Wprowadzenie do NLP i zastosowań

✅ Narzędzia: NLTK, spaCy, Hugging Face, OpenAI

✅ Podstawowe operacje: tokenizacja, lematyzacja, POS tagging

✅ Praktyczne projekty: sentiment analysis, spam detection

### Dzień 2 - Zaawansowane:
✅ Architektury Transformer i modele BERT, GPT, T5

✅ Fine-tuning modeli

✅ Generowanie i rozumienie tekstu

✅ Zastosowania biznesowe: chatboty, automatyzacja, personalizacja

## Best Practices dla produkcji:

### 1. Wybór modelu
- Balansuj między jakością a szybkością
- Rozważ DistilBERT dla szybszych aplikacji
- Użyj cache'owania dla często używanych predykcji

### 2. Preprocessing
- Standaryzuj pipeline preprocessing
- Obsługuj edge cases (emoji, URL, HTML)
- Waliduj dane wejściowe

### 3. Monitoring
- Loguj wszystkie predykcje
- Monitoruj drift modelu
- Zbieraj feedback użytkowników

### 4. Bezpieczeństwo
- Nie wysyłaj wrażliwych danych do zewnętrznych API
- Implementuj rate limiting
- Waliduj i sanitizuj output modelu

### 5. Skalowalność
- Użyj batch processing gdzie możliwe
- Rozważ GPU dla inference
- Implementuj kolejki dla asynchronicznego przetwarzania

## Dalsze kroki:

### Zasoby do nauki:
- 📚 Hugging Face Course: https://huggingface.co/course
- 📚 Fast.ai NLP: https://www.fast.ai/
- 📚 Papers With Code: https://paperswithcode.com/

### Community:
- 💬 Hugging Face Discord
- 💬 r/MachineLearning na Reddit
- 💬 AI/ML meetupy lokalne

### Projekty do praktyki:
1. Zbuduj własnego chatbota
2. Stwórz system rekomendacji treści
3. Fine-tunuj model na własnych danych
4. Zaimplementuj multilingual search

---

**Gratulacje! 🎉 Ukończyłeś/aś szkolenie z NLP!**

**Powodzenia w Twoich projektach AI! 🚀**